In [197]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from pca import PCA
import pandas as pd

data_balanced = pd.read_csv('data_balanced.csv')    
# Assuming data_balanced is a DataFrame with 'class' as the target
X = data_balanced.drop(columns=['class'])  # Features
y = data_balanced['class']  # Target

# Convert to NumPy arrays
# X = np.asarray(X)
# y = np.asarray(y)
y= y.astype('int')

# Step 1: Apply EFSA for feature selection
# efsa = EFSA(correlation_threshold=0.4, n_features=5)  # Adjust thresholds as needed
X_selected = X  # efsa.fit_transform(X, y)


# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# Split into train and test sets (before feature selection)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Step 2: Feature Scaling (optional, depending on your model)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Step 3: Apply PCA for dimensionality reduction
# pca = PCA(n_components=5)  # Adjust number of components as needed
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_test_pca = pca.transform(X_test_scaled)


In [198]:
from efsa import EFSA


efsa = EFSA(correlation_threshold=0.43)
selected,_ = efsa.correlation_selection(X_train.values, y_train.values)
# print(selected)
# X_selected = X_train[:,selected]  

#read the first element of each tuple in the list
selected = [i[0] for i in selected]
if selected[0]==0:
    selected = selected[1:]

#get the all rows and only the selected columns
X_selected = X_train.iloc[:,selected]
X_test_selected = X_test.iloc[:,selected]

# X_selected = X_train.iloc[:,selected]


# remove the column 0 is present 
# if 0 in selected.columns:
    # X_selected = X_selected.drop(columns=[0])
# X_test = efsa.transform(X_test.values)
print(len(selected))
print(selected)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)



7
[58, 132, 133, 135, 404, 620, 656]


In [199]:
# from logistic import Logistic
from sklearn.preprocessing import StandardScaler
from logistic import Logistic

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logistic = Logistic(learning_rate=0.1, epochs=1000)
logistic.one_vs_all(X_train_scaled, y_train)
y_pred = logistic.predict_multiclass(X_test_scaled)
print('Accuracy:', accuracy_score(y_test, y_pred))

Epoch 0: Loss = 0.6931471805599454
Epoch 100: Loss = 0.1224122531534165
Epoch 200: Loss = 0.08346340780761291
Epoch 300: Loss = 0.06444729638134768
Epoch 400: Loss = 0.0526557351037139
Epoch 500: Loss = 0.04452936217650493
Epoch 600: Loss = 0.038568893659783214
Epoch 700: Loss = 0.03400599298811872
Epoch 800: Loss = 0.03040006703764631
Epoch 900: Loss = 0.027478885916072917
Epoch 0: Loss = 0.6931471805599454
Epoch 100: Loss = 0.12241225315341649
Epoch 200: Loss = 0.08346340780761292
Epoch 300: Loss = 0.06444729638134766
Epoch 400: Loss = 0.0526557351037139
Epoch 500: Loss = 0.04452936217650493
Epoch 600: Loss = 0.038568893659783214
Epoch 700: Loss = 0.03400599298811872
Epoch 800: Loss = 0.03040006703764631
Epoch 900: Loss = 0.027478885916072924
Accuracy: 0.8908554572271387


In [200]:

from decision_tree import DecisionTree

dt = DecisionTree(max_depth=3)
dt.fit(X_selected, y_train)
y_pred = dt.predict(X_test_selected)
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.7345132743362832


In [ ]:
from RF import RandomForest
rf = RandomForest(n_trees=3, max_depth=3)
X_selected_pd = pd.DataFrame(X_selected)
rf.fit(X_selected_pd, y_train)
y_pred = rf.predict(X_test_selected)
print('Accuracy:', accuracy_score(y_test, y_pred))

Fitting tree 1...
Fitting tree 2...
Fitting tree 3...
Accuracy: 0.7669616519174042


c:\Users\pc\OneDrive\Documents\IASD\machine learning\PROJET\decision_tree.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_value = example[tree.feature]


In [210]:
from lightGBM_copy import LightFBM
# Train a LightGBM model
lgbm = LightFBM()
lgbm.fit(X_selected.values, y_train.values)
y_pred = lgbm.predict(X_test_selected.values)
y_pred = (y_pred >= 0.5).astype(int)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(f"accuracy_score : {accuracy_score(y_test, y_pred)}")


Fitting tree 1
Fitting tree 2
Fitting tree 3
accuracy_score : 0.7817109144542773
